### imports

In [1]:
import os
import sys

### Function Definitions

In [2]:
# Function to print lines between min_ and max_ values
def print_some_lines(line_list, min_, max_):
    j = 0
    for i in madx_simulation_lines:
        if (j <= max_) and (j >= min_):
            print i
        j = j + 1
    return

print_all_lines = lambda x: sys.stdout.write("\n".join(x) + "\n")


# PTC-PyORBIT Examples: Create a PTC Flat File using a MAD-X lattice description

We obtain our accelerator lattice definition in MAD-X format from the CERN BE-ABP maintained acc-models GitLab repository: https://gitlab.cern.ch/acc-models

Each machine has its own repository. Please make sure to use the latest lattice relevent to your specific requirements.

## Step 1: RF Cavity definition modification: no_cavity_totalpath flag

The first thing we must to is check/edit lattice settings that will create an unusable PTC flat file.

We must use the no_cavity_totalpath flag and set used RF cavity lengths to 1.0 m. In this case all RF cavities are removed in the PS_Lattice/remove_elements.seq file, so we only edit one line.

In [3]:
# Open the file with read only permit
harmonic7_sequence_file = open('PS_Lattice/ps_ss_h7.seq', "r")

# use readlines to read all lines in the file
# The variable "harmonic7_sequence_lines" is a list containing all lines in the file
harmonic7_sequence_lines = harmonic7_sequence_file.readlines()

# close the file after reading the lines
harmonic7_sequence_file.close()

We replace line 78:

In [4]:
print harmonic7_sequence_lines[77]

!ACC10     : RFCAVITY    , L := 2.45;      ! 10 MHz radio frequency cavity



With:

In [5]:
print harmonic7_sequence_lines[78]

ACC10     : RFCAVITY    , L := 1.0, LAG=0, HARMON=7, no_cavity_totalpath; ! Fix for PTC PyORBIT SC sims



The specific cavity used is defined in line 161:

In [6]:
print harmonic7_sequence_lines[160]

 PA.C10.11      : ACC10     , AT = 1.509, SLOT_ID = 2253478;



Note that as we have a harmonic 9 sequence file (PS_Lattice/ps_ss_h9.seq) this step is repeated for that file also.

## Step 2: MAD-X Setup

Our MAD-X simulation is defined using the Create_PTC_flat_file.madx script.

Brief file description:

In [7]:
madx_simulation_file = open('Create_PTC_flat_file.madx', "r")
madx_simulation_lines = madx_simulation_file.readlines()
madx_simulation_file.close()

In [15]:
print_some_lines(madx_simulation_lines, 0, 24)

TITLE, "Flat file generation for PTC-PyORBIT";

/***********************************************************************

 *                  CERN Proton Synchrotron: Injection

 *        Create PTC Flat File for PTC-PyORBIT lattice definition

 *

 * Last Edit: 11.06.20 Haroon Rafique CERN BE-ABP-HSI

 **********************************************************************/





/***********************************************************************

 *                       USER SETTINGS FLAGS

 *

 *      Here we have the necessary files and macros for:

 *      ------------------------------------------------

 *      - Run2 and Run3, LHC BCMS and Standard beams.

 *      - PS Injection settings NOT including injection bump.

 *      - Addition of single or distributed quadrupolar error.

 *      - Matching of tunes using Low Energy Quadrupoles (LEQs) or 

 *        Pole Face Windings (PFWs).

 *      - Matching of chromaticity using PFWs.

 *      - Changing the lattice start posi

***
In this example we will use the Run 3 (LIU) Standard (harmonic = 7) beam, with no errors, using the bare machine tune and chromaticity. This is set using the flags in the next part of Create_PTC_flat_file.madx

In [18]:
print_some_lines(madx_simulation_lines, 25, 35)

quad_error = 0;	                ! Add quadrupolar error on QDN72

distributed_quad_error = 0;	! Add quadrupolar error on all QDNs



match_tunes = 0;                ! Match tunes to values in tunes.str

use_pfw = 0;	                ! Match tunes with PFW (1) or LEQ (0)

match_chroma = 0;               ! Match chromaticity to given values



BCMS = 0;		!Harmonic 9 for BCMS, 7 for Standard LHC beams

LIU = 1;		!LIU (1) KE = 2GeV, pre-LIU (0) KE = 1.4GeV



start_lattice_marker = 0;



## Step 3: Run MAD-X 

In [20]:
madx_command = '../madx-linux64_v5_02_00 < Create_PTC_flat_file.madx'
os.system(madx_command)

0

In [10]:
print_all_lines(madx_simulation_lines)

TITLE, "Flat file generation for PTC-PyORBIT";

/***********************************************************************

 *                  CERN Proton Synchrotron: Injection

 *        Create PTC Flat File for PTC-PyORBIT lattice definition

 *

 * Last Edit: 11.06.20 Haroon Rafique CERN BE-ABP-HSI

 **********************************************************************/





/***********************************************************************

 *                       USER SETTINGS FLAGS

 *

 *      Here we have the necessary files and macros for:

 *      ------------------------------------------------

 *      - Run2 and Run3, LHC BCMS and Standard beams.

 *      - PS Injection settings NOT including injection bump.

 *      - Addition of single or distributed quadrupolar error.

 *      - Matching of tunes using Low Energy Quadrupoles (LEQs) or 

 *        Pole Face Windings (PFWs).

 *      - Matching of chromaticity using PFWs.

 *      - Changing the lattice start posi

In [ ]:
import numpy as np
import matplotlib.cm as cm
from math import log10, floor
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from scipy.optimize import curve_fit

### Figure parameters

In [ ]:
plt.rcParams['figure.figsize'] = [5.0, 5.0]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

plt.rcParams['font.size'] = 10
plt.rcParams['legend.fontsize'] = 'small'
plt.rcParams['lines.linewidth'] = 1.5

### Function Declarations

In [ ]:
def harmonic_fn(x, a, b, c, d):
    return a + b*np.cos(c + 2*np.pi*d*x)

def quadratic(x, a, m, c):
    return a*x**2 + x*m + c

def quadratic(x, a, m, c):
    return a*x**2 + x*m + c

def Md_fit(x, a, b, c, d):
        return a + b*np.cos(c*x+d)
    
def Md_ramp_fit(x, a, b, c, d):
    return a + b*(x-1)*np.cos(c*(x-1)+d)

def round_sig(x, sig=4):
    return round(x, sig-int(floor(log10(abs(x))))-1)

In [ ]:
def add_input_file(dd, filename, label):
	f = filename
	p = dict()
	sio.loadmat(f, mdict=p)
	dd[label] = p	
	print '\tAdded output data from ', filename, '\t dictionary key: ', label
	return dd

## Import Data

In [ ]:
#Horizontal Scan No Space Charge

# Create dd dictionary
dd3 = dict()

dd3 = add_input_file(dd3, 'Run2_Standard/output/output.mat', 'Run2 Std')
dd3 = add_input_file(dd3, 'Run2_BCMS/output/output.mat', 'Run2 BCMS')
dd3 = add_input_file(dd3, 'Run3_Standard_2021/output/output.mat', 'LIU Std 2021')
dd3 = add_input_file(dd3, 'Run3_Standard_2022/output/output.mat', 'LIU Std 2022')
dd3 = add_input_file(dd3, 'Run3_Standard_2023/output/output.mat', 'LIU Std 2023')
dd3 = add_input_file(dd3, 'Run3_BCMS/output/output.mat', 'LIU BCMS')

### Plot Parameters

In [ ]:
save_folder = 'Plots/'

sc = 'SbS'
legend_label = 'Case' # r'$\frac{\delta p}{p_0}$'
main_label = 'PS_Injection_IB_5%'
main_label2 = main_label + '_zoom'
scaled_label = main_label + '_scaled'
turn_tot = 2200
zoom_turns = 50
betagamma = 2.492104532 * 0.9159915293879255

In [ ]:
dd3['Run2 Std'].keys()

In [ ]:
parameter = 'turn_duration'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r't$_{turn}$ [s]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0], label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1)
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'intensity'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel('I [protons]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*1000, label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1)
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'n_mp'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$N_{macroparticles}$ [-]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*1000, label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1)
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'sig_x'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$\sigma_x$ [mm]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*1000, label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1)
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'sig_y'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$\sigma_y$ [mm]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*1000, label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1)
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'dpp_rms'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$\frac{\delta p}{p_0}_{RMS}$ [MeV]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*1E3, label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'bunchlength'
multi1 = 1E9

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$B_l$ [ns]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):  
        av_ = np.mean(dd3[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd3[key][parameter][0]*multi1) - np.min(dd3[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'    
        lab_ = key + ' ' + str(round_sig(rang_,3)) + ' ns'           
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*multi1, label=lab_, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'beta_x'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$\beta_{x}$ [m]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0], label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'beta_y'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$\beta_{x}$ [m]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0], label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'epsn_x'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$\epsilon_{x_{RMS}}$ [mm mrad]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*1E6, label=key, color=colors[c_it]);
        av_ = np.mean(dd3[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd3[key][parameter][0]*multi1) - np.min(dd3[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'    
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'epsn_y'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$\epsilon_{y_{RMS}}$ [mm mrad]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0]*1E6, label=key, color=colors[c_it]);
        av_ = np.mean(dd3[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd3[key][parameter][0]*multi1) - np.min(dd3[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'    
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter1 = 'epsn_x'
multi1 = 1E6
parameter2 = 'epsn_y'
multi2 = 1E6

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' average emittance'
ax1.set_title(tit);

ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        x = np.array(dd3[key]['turn'][0])
        y = np.array((dd3[key][parameter1][0]*multi1 + dd3[key][parameter2][0]*multi2)/2)
        ax1.plot(x, y, label=key, color=colors[c_it]);
        av_ = np.mean(y[-100])
        rang_ = np.abs(np.max(y) - np.min(y))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'    
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_average_emittance.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'D_x'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$D_{x}$ [m]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0], label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

In [ ]:
parameter = 'D_y'

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$D_{y}$ [m]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd3.keys())))
c_it = int(0)

for key, value in sorted(dd3.iteritems()):
        ax1.plot(dd3[key]['turn'][0], dd3[key][parameter][0], label=key, color=colors[c_it]);
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ main_label + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

# Compare individual cases

## Run2 Standard

In [ ]:
# Create dd dictionary
dd = dict()

run_lab = 'Run2_Std'

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', 'Run2 Std')
dd = add_input_file(dd, '../19_SC_Mini/Run2_Standard/output/output.mat', 'Run2 Std Old')

In [ ]:
parameter = 'bunchlength'
multi1 = 1E9

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$B_l$ [ns]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd.keys())))
c_it = int(0)

for key, value in sorted(dd.iteritems()):
        av_ = np.mean(dd[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd[key][parameter][0]*multi1) - np.min(dd[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%' 
        lab_ = key + ' ' + str(round_sig(rang_,3)) + ' ns'
        ax1.plot(dd[key]['turn'][0], dd[key][parameter][0]*multi1, label=lab_, color=colors[c_it]);
   
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ run_lab + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

## Run 2 BCMS

In [ ]:
# Create dd dictionary
dd = dict()
run_lab = 'Run2_BCMS'
dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', 'Run2 BCMS')
dd = add_input_file(dd, '../19_SC_Mini/Run2_BCMS/output/output.mat', 'Run2 BCMS Old')

In [ ]:
parameter = 'bunchlength'
multi1 = 1E9

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$B_l$ [ns]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd.keys())))
c_it = int(0)

for key, value in sorted(dd.iteritems()):
        av_ = np.mean(dd[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd[key][parameter][0]*multi1) - np.min(dd[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%' 
        lab_ = key + ' ' + str(round_sig(rang_,3)) + ' ns'
        ax1.plot(dd[key]['turn'][0], dd[key][parameter][0]*multi1, label=lab_, color=colors[c_it]);
   
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ run_lab + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

## Run 3 BCMS

In [ ]:
# Create dd dictionary
dd = dict()
run_lab = 'Run3_BCMS'
dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', 'Run3 BCMS')
dd = add_input_file(dd, '../19_SC_Mini/Run3_BCMS/output/output.mat', 'Run3 BCMS Old')

In [ ]:
parameter = 'bunchlength'
multi1 = 1E9

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$B_l$ [ns]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd.keys())))
c_it = int(0)

for key, value in sorted(dd.iteritems()):
        av_ = np.mean(dd[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd[key][parameter][0]*multi1) - np.min(dd[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%' 
        lab_ = key + ' ' + str(round_sig(rang_,3)) + ' ns'
        ax1.plot(dd[key]['turn'][0], dd[key][parameter][0]*multi1, label=lab_, color=colors[c_it]);
   
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ run_lab + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

# Run 3 Std 2021

In [ ]:
# Create dd dictionary
dd = dict()
run_lab = 'Run3_Std_2021'
dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', 'Run3 Std 2021')
dd = add_input_file(dd, '../19_SC_Mini/Run3_Standard_2021/output/output.mat', 'Run3 Std 2021 Old')

In [ ]:
parameter = 'bunchlength'
multi1 = 1E9

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$B_l$ [ns]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd.keys())))
c_it = int(0)

for key, value in sorted(dd.iteritems()):
        av_ = np.mean(dd[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd[key][parameter][0]*multi1) - np.min(dd[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%' 
        lab_ = key + ' ' + str(round_sig(rang_,3)) + ' ns'
        ax1.plot(dd[key]['turn'][0], dd[key][parameter][0]*multi1, label=lab_, color=colors[c_it]);
   
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ run_lab + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

# Run 3 Std 2022

In [ ]:
# Create dd dictionary
dd = dict()
run_lab = 'Run3_Std_2022'
dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', 'Run3 Std 2022')
dd = add_input_file(dd, '../19_SC_Mini/Run3_Standard_2022/output/output.mat', 'Run3 Std 2022 Old')

In [ ]:
parameter = 'bunchlength'
multi1 = 1E9

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$B_l$ [ns]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd.keys())))
c_it = int(0)

for key, value in sorted(dd.iteritems()):
        av_ = np.mean(dd[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd[key][parameter][0]*multi1) - np.min(dd[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%' 
        lab_ = key + ' ' + str(round_sig(rang_,3)) + ' ns'
        ax1.plot(dd[key]['turn'][0], dd[key][parameter][0]*multi1, label=lab_, color=colors[c_it]);
   
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ run_lab + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

# Run 3 Std 2023

In [ ]:
# Create dd dictionary
dd = dict()
run_lab = 'Run3_Std_2023'
dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', 'Run3 Std 2023')
dd = add_input_file(dd, '../19_SC_Mini/Run3_Standard_2023/output/output.mat', 'Run3 Std 2023 Old')

In [ ]:
parameter = 'bunchlength'
multi1 = 1E9

fig1 = plt.figure(facecolor='w', edgecolor='k')
ax1 = fig1.add_subplot(111)

tit = main_label + ' ' + sc + ' ' + parameter
ax1.set_title(tit);

ax1.set_ylabel(r'$B_l$ [ns]');
ax1.set_xlabel('Turn [-]');

colors = cm.rainbow(np.linspace(0, 1, len(dd.keys())))
c_it = int(0)

for key, value in sorted(dd.iteritems()):
        av_ = np.mean(dd[key][parameter][0][-100]*multi1)
        rang_ = np.abs(np.max(dd[key][parameter][0]*multi1) - np.min(dd[key][parameter][0]*multi1))
        print key, 'Average ', parameter, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%' 
        lab_ = key + ' ' + str(round_sig(rang_,3)) + ' ns'
        ax1.plot(dd[key]['turn'][0], dd[key][parameter][0]*multi1, label=lab_, color=colors[c_it]);
   
        c_it = c_it + 1;
        
ax1.legend(loc=1);
ax1.grid(lw=1, ls=':');
ax1.set_xlim(-1,turn_tot)

savename = save_folder + '/'+ run_lab + '_' + parameter + '.png'
plt.savefig(savename, dpi = 800);

# Plots for IPP talk

In [ ]:
plt.rcParams['figure.figsize'] = [5.0, 4.5]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 14

plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

plt.rcParams['font.size'] = 10
plt.rcParams['legend.fontsize'] = 12

plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 5

In [ ]:
dd = dict()

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', '00 Run2 Standard')
#dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', '01 Run2 BCMS')

#dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', '02 LIU BCMS')

#dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', '03 LIU Standard 2021')
#dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', '04 LIU Standard 2022')
#dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', '05 LIU Standard 2023')

parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
#ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)








parameter1 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_y$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)









parameter1 = 'epsn_x'
parameter2 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_average_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], ((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)






parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'_legend.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)

In [ ]:
dd = dict()

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', '00 Run2 Standard')
dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', '01 Run2 BCMS')

#dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', '02 LIU BCMS')

#dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', '03 LIU Standard 2021')
#dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', '04 LIU Standard 2022')
#dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', '05 LIU Standard 2023')

parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
#ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)








parameter1 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_y$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)









parameter1 = 'epsn_x'
parameter2 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_average_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], ((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)






parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'_legend.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)

In [ ]:
dd = dict()

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', '00 Run2 Standard')
dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', '01 Run2 BCMS')
dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', '02 LIU Standard')

#dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', '02 LIU BCMS')

#dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', '03 LIU Standard 2021')
#dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', '04 LIU Standard 2022')

parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
#ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)








parameter1 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_y$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)









parameter1 = 'epsn_x'
parameter2 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_average_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], ((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)






parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'_legend.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)

In [ ]:
dd = dict()

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', '00 Run2 Standard')
dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', '01 Run2 BCMS')
dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', '02 LIU Standard')
dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', '03 LIU BCMS')

#dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', '03 LIU Standard 2021')
#dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', '04 LIU Standard 2022')

parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
#ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)








parameter1 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_y$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)









parameter1 = 'epsn_x'
parameter2 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_average_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], ((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)






parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_' + str(len(dd))+'_legend.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)

In [ ]:
dd = dict()

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', '00 Run2 Standard')
dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', '01 Run2 BCMS')

dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', '02 LIU BCMS')

dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', '03 LIU Standard 2021')
#dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', '04 LIU Standard 2022')
#dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', '05 LIU Standard 2023')

parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
#ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)








parameter1 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_y$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)









parameter1 = 'epsn_x'
parameter2 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_average_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], ((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)






parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'_legend.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)

In [ ]:
dd = dict()

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', '00 Run2 Standard')
dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', '01 Run2 BCMS')

dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', '02 LIU BCMS')

dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', '03 LIU Standard 2021')
dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', '04 LIU Standard 2022')
#dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', '05 LIU Standard 2023')

parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
#ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)








parameter1 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_y$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)









parameter1 = 'epsn_x'
parameter2 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_average_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], ((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)






parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'_legend.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)

In [ ]:
dd = dict()

dd = add_input_file(dd, 'Run2_Standard/output/output.mat', '00 Run2 Standard')
dd = add_input_file(dd, 'Run2_BCMS/output/output.mat', '01 Run2 BCMS')

dd = add_input_file(dd, 'Run3_BCMS/output/output.mat', '02 LIU BCMS')

dd = add_input_file(dd, 'Run3_Standard_2021/output/output.mat', '03 LIU Standard 2021')
dd = add_input_file(dd, 'Run3_Standard_2022/output/output.mat', '04 LIU Standard 2022')
dd = add_input_file(dd, 'Run3_Standard_2023/output/output.mat', '05 LIU Standard 2023')

parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
#ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)








parameter1 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_y$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    print key, 'Average ', parameter1, ' = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)









parameter1 = 'epsn_x'
parameter2 = 'epsn_y'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_average_rampup_' + str(len(dd))+'.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\frac{ \epsilon_{x_{RMS}} + \epsilon_{y_{RMS}} }{2}$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_y'][0][-500:]) - dd[key]['epsn_y'][0][0])/(dd[key]['epsn_y'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    lab_ = key[3:] + r' $\frac{\Delta \epsilon_y}{\epsilon_{y_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'

    ax1.plot(dd[key]['turn'][0], ((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2, color=colors[c_it], label=lab_);
    
    start_ = ((dd[key][parameter1][0][0]+dd[key][parameter2][0][0])*multi1)/2
    fin_ = ((dd[key][parameter1][0][-1]+dd[key][parameter2][0][-1])*multi1)/2
    av_ = np.mean(((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2)
    rang_ = np.abs(np.max(((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2) - np.min(((dd[key][parameter1][0]+dd[key][parameter2][0])*multi1)/2))
    
    
    print key, 'Average average epsn = ' , \
        str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%',\
        '\n\t growth = ', str(round_sig(((fin_-start_)*100/start_) ,3)), '%'
    c_it = c_it + 1;

#ax1.legend(loc=10, framealpha=0.0)
ax1.grid(which='both', ls=':', lw=0.5)

ax1.set_ylim(1.0, 2.4)
plt.tight_layout()
fig.savefig(savename)






parameter1 = 'epsn_x'
multi1 = 1E6

fig, ax1 = plt.subplots();
#ax1.set_title('PyORBIT Output: BLonD Input Test')

savename = save_folder +'/' + main_label + '_' +  parameter1 + '_rampup_' + str(len(dd))+'_legend.png'

ax1.set_xlabel('Turn [-]')
ax1.set_ylabel(r'$\epsilon_x$ [mm mrad]');


colors = cm.rainbow(np.linspace(0, 1, 7))
c_it = int(0)

for key in sorted(dd.iterkeys()):
    eps_ov_eps_0 = 100*(np.mean(dd[key]['epsn_x'][0][-500:]) - dd[key]['epsn_x'][0][0])/(dd[key]['epsn_x'][0][0])
    # eps_ov_eps_0 = 100*(np.max(dd1[key]['epsn_x'][0]) - np.min(dd1[key]['epsn_x'][0]))/(dd1[key]['epsn_x'][0][0])
    #lab_ = key[3:] + r' $\frac{\Delta \epsilon_x}{\epsilon_{x_0}} = $' +str(round_sig(eps_ov_eps_0,2))+'%'
    lab_ = key[3:]
    ax1.plot(dd[key]['turn'][0], dd[key][parameter1][0]*multi1, color=colors[c_it], label=lab_);
    
    #av_ = np.mean(dd[key][parameter1][0][-100]*multi1)
    #rang_ = np.abs(np.max(dd[key][parameter1][0]*multi1) - np.min(dd[key][parameter1][0]*multi1))
    
    #print key, 'Average ', parameter1, ' = ' , \
    #    str(round_sig(av_)), ' range = ', round_sig(rang_), ' = ', round_sig(rang_/av_)*100, '%'
    c_it = c_it + 1;

ax1.set_ylim(1.0, 2.4)
ax1.legend(loc=10, framealpha=1.0)
ax1.grid(which='both', ls=':', lw=0.5)

plt.tight_layout()
fig.savefig(savename)